In [1]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [2]:
#best_model = YOLO("best_model.pt")

#print(f"The task type of the model is: {best_model.task}")

In [2]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [10]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("/home/under1/Detect/jeongui/no_val/sagittal_sharpening/runs/detect/train/train_sharp/weights/best.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=34720) New https://pypi.org/project/ultralytics/8.2.67 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=34723) Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=34723) engine/trainer: task=detect, mode=train, model=/home/under1/Detect/jeongui/no_val/sagittal_sharpening/runs/detect/train/train_sharp/weights/best.pt, data=/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=3

  0%|          | 0.00/6.23M [00:00<?, ?B/s]
  3%|▎         | 192k/6.23M [00:00<00:03, 1.94MB/s]
 98%|█████████▊| 6.08M/6.23M [00:01<00:00, 4.17MB/s]
100%|██████████| 6.23M/6.23M [00:01<00:00, 4.08MB/s]
100%|██████████| 6.23M/6.23M [00:01<00:00, 3.67MB/s]
 73%|███████▎  | 4.56M/6.23M [00:01<00:00, 4.10MB/s]
100%|██████████| 6.23M/6.23M [00:02<00:00, 3.08MB/s]
100%|██████████| 6.23M/6.23M [00:02<00:00, 2.97MB/s]


(_tune pid=34722) AMP: checks passed ✅
(_tune pid=34722) Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB) [repeated 3x across cluster]
(_tune pid=34722) engine/trainer: task=detect, mode=train, model=/home/under1/Detect/jeongui/no_val/sagittal_sharpening/runs/detect/train/train_sharp/weights/best.pt, data=/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None,

train: Scanning /home/under1/Detect/jeongui/no_val/sagittal_sharpening/labels/train.cache... 264 images, 0 backgrounds, 0 corrupt: 100%|██████████| 264/264 [00:00<?, ?it/s]
  0%|          | 0.00/6.23M [00:00<?, ?B/s] [repeated 3x across cluster]
 82%|████████▏ | 5.10M/6.23M [00:02<00:00, 4.55MB/s] [repeated 63x across cluster]


(_tune pid=34722) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/3 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 1/3 [00:00<00:00,  3.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [00:00<00:00,  2.85it/s]
    200/200       2.2G    0.01248      1.694      1.098         12        640:  88%|████████▊ | 15/17 [00:01<00:00,  8.33it/s] [repeated 13x across cluster]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.74it/s]


(_tune pid=34721)                    all         70         77      0.478      0.703      0.577      0.376
(_tune pid=34721)               negative         70         39      0.505      0.564      0.527      0.349
(_tune pid=34721)               positive         70         38      0.451      0.842      0.627      0.403


INFO:tensorboardX.summary:Summary name ray/tune/metrics/precision(B) is illegal; using ray/tune/metrics/precision_B_ instead.
INFO:tensorboardX.summary:Summary name ray/tune/metrics/recall(B) is illegal; using ray/tune/metrics/recall_B_ instead.
INFO:tensorboardX.summary:Summary name ray/tune/metrics/mAP50(B) is illegal; using ray/tune/metrics/mAP50_B_ instead.
INFO:tensorboardX.summary:Summary name ray/tune/metrics/mAP50-95(B) is illegal; using ray/tune/metrics/mAP50-95_B_ instead.


(_tune pid=34721) Speed: 0.3ms preprocess, 1.1ms inference, 0.0ms loss, 2.7ms postprocess per image
(_tune pid=34721) Results saved to runs/detect/tune/tune_raytune


INFO:tensorboardX.summary:Summary name ray/tune/metrics/precision(B) is illegal; using ray/tune/metrics/precision_B_ instead.
INFO:tensorboardX.summary:Summary name ray/tune/metrics/recall(B) is illegal; using ray/tune/metrics/recall_B_ instead.
INFO:tensorboardX.summary:Summary name ray/tune/metrics/mAP50(B) is illegal; using ray/tune/metrics/mAP50_B_ instead.
INFO:tensorboardX.summary:Summary name ray/tune/metrics/mAP50-95(B) is illegal; using ray/tune/metrics/mAP50-95_B_ instead.
2024-07-28 12:34:07,849	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/sagittal/runs/detect/tune/_tune_2024-07-28_12-13-29' in 0.0144s.
2024-07-28 12:34:07,866	INFO tune.py:1041 -- Total run time: 1237.89 seconds (1237.83 seconds for the tuning loop).


In [11]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.05488586225769321, 'lrf': 0.7180374727086953, 'momentum': 0.8290500829072247, 'weight_decay': 0.0005448831829968968, 'warmup_epochs': 2.1182739966945237, 'warmup_momentum': 0.6135994074133233, 'box': 0.09876569802728466, 'cls': 3.588737402971903, 'hsv_h': 0.09636627605010294, 'hsv_s': 0.3450973669431999, 'hsv_v': 0.7125525342743981, 'degrees': 23.800271388880702, 'translate': 0.5112401049845391, 'scale': 0.833036974463395, 'shear': 0.7103605819788694, 'perspective': 8.712929970154071e-05, 'flipud': 0.02021839744032572, 'fliplr': 0.832619845547938, 'bgr': 0.7781567509498505, 'mosaic': 0.8700121482468192, 'mixup': 0.978618342232764, 'copy_paste': 0.7991585642167236, 'data': '/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.36981, 'metrics/recall(B)': 0.59953, 'metrics/mAP50(B)': 0.43294, 'metrics/mAP50-95(B)': 0.19174, 'val/box_loss': 0

In [12]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.07805511233688268, 'lrf': 0.1270916816102439, 'momentum': 0.843169988104459, 'weight_decay': 0.0001433532874090464, 'warmup_epochs': 4.7233445852479194, 'warmup_momentum': 0.49575590566256805, 'box': 0.09463914919829426, 'cls': 1.2053113259975823, 'hsv_h': 0.07742336894342167, 'hsv_s': 0.4105352989948937, 'hsv_v': 0.5115905539817837, 'degrees': 0.8455410196359814, 'translate': 0.5558719473682894, 'scale': 0.5508861504501793, 'shear': 6.169339968747569, 'perspective': 0.0009437480785146242, 'flipud': 0.6818202991034834, 'fliplr': 0.359507900573786, 'bgr': 0.43703195379934145, 'mosaic': 0.6976311959272649, 'mixup': 0.06022547162926983, 'copy_paste': 0.6667667154456677, 'data': '/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}


In [13]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml",
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

New https://pypi.org/project/ultralytics/8.2.67 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/no_val/sagittal_sharpening/sharpening.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, clas

train: Scanning /home/under1/Detect/jeongui/no_val/sagittal_sharpening/labels/train.cache... 264 images, 0 backgrounds, 0 corrupt: 100%|██████████| 264/264 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/home/under1/.conda/envs/yolo/lib/python3.10/site-packages/albumentations/core/composition.py:147: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning /home/under1/Detect/jeongui/no_val/sagittal_sharpening/labels/test.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.07805511233688268' and 'momentum=0.843169988104459' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0001433532874090464), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    171/200      2.11G    0.01695      2.593      1.294         12        640: 100%|██████████| 17/17 [00:01<00:00, 12.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00, 10.90it/s]

                   all         70         77      0.362       0.77      0.452      0.273
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 71, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



171 epochs completed in 0.111 hours.
Optimizer stripped from runs/detect/train/train_raytune/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_raytune/weights/best.pt, 6.3MB

Validating runs/detect/train/train_raytune/weights/best.pt...
Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  9.27it/s]


                   all         70         77      0.455      0.806      0.524      0.365
              negative         70         39      0.433      0.769      0.425      0.297
              positive         70         38      0.477      0.842      0.623      0.434
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train/train_raytune


In [14]:
best_results.results_dict

{'metrics/precision(B)': 0.4549388028418082,
 'metrics/recall(B)': 0.8056680161943319,
 'metrics/mAP50(B)': 0.5241237803898247,
 'metrics/mAP50-95(B)': 0.36512495601291806,
 'fitness': 0.38102483845060875}

In [15]:
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/sagittal/runs/detect/train/train_raytune/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/jeongui/no_val/sagittal_sharpening/labels/test.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  6.13it/s]


                   all         70         77      0.455      0.806      0.524      0.364
              negative         70         39      0.434      0.769      0.425      0.295
              positive         70         38      0.476      0.842      0.623      0.434
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val/val_raytune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fda4fbc17b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [9]:
# test

experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/jeongui/sagittal/sharpening/labels/test.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  4.11it/s]


                   all         70         77      0.407      0.692      0.496      0.325
              negative         70         39      0.429      0.462      0.469      0.316
              positive         70         38      0.384      0.921      0.523      0.335
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/test/test_raytune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd990400c40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804